In [1]:
import qiskit
import qiskit_superstaq as qss

from general_superstaq import ResourceEstimate
from qiskit.circuit.random import random_circuit

# Getting Resource Estimate Costs

In [2]:
# Connect to SuperstaQ
# Provide your api key to the using the "api_key" argument if
# SUPERSTAQ_API_KEY environment variable is not set.
superstaq = qss.superstaq_provider.SuperstaQProvider()

# Resource Estimate for Simple Circuit

In [3]:
# Create simple circuit and submit to Neutral Atom Device
circuit = qiskit.QuantumCircuit(2)
circuit.cnot(0, 1)
circuit.h(1)

circuit.draw()

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ H ├
     └───┘└───┘

In [4]:
resource_estimate = superstaq.resource_estimate(circuit, "neutral_atom_qpu")
print(resource_estimate)

ResourceEstimate(num_single_qubit_gates=1, num_two_qubit_gates=1, depth=2)


# Resource Estimate for More Complex Random Circuit

In [5]:
# Create and submit random circuit to Neutral Atom Device
circuit_random = random_circuit(4, 4, measure=True)
circuit_random.draw()

»
q_0: ─■─────────────────────────────────────────────────────■──────────────»
      │                       ┌───┐            ┌────────────┴────────────┐ »
q_1: ─┼───────────────────────┤ H ├────────────┤ U3(4.4986,5.217,5.8575) ├─»
      │U1(3.5116) ┌───────────┴───┴───────────┐├─────────────────────────┴┐»
q_2: ─■───────────┤ U3(1.1229,0.48484,1.7987) ├┤ U3(2.1973,6.0012,4.1863) ├»
         ┌───┐    └──────────┬─────┬──────────┘└──────────────────────────┘»
q_3: ────┤ X ├───────────────┤ Sdg ├───────────────────────────────────────»
         └───┘               └─────┘                                       »
c: 4/══════════════════════════════════════════════════════════════════════»
                                                                           »
«     ┌───┐             ┌─┐      
«q_0: ┤ H ├──────X──────┤M├──────
«     └─┬─┘┌───┐ │ ┌───┐└╥┘┌─┐   
«q_1: ──┼──┤ T ├─┼─┤ S ├─╫─┤M├───
«       │  ├───┤ │ └┬─┬┘ ║ └╥┘   
«q_2: ──┼──┤ T ├─┼──┤M├──╫──╫────
«       │  └───┘ │  └╥┘  ║  ║ ┌─┐
«q_3: ──■────────X───╫───╫──╫─┤M├
«                    ║   ║  ║ └╥┘
«c: 4/═══════════════╩═══╩══╩══╩═
«                    2   0  1  3

In [6]:
resource_estimate = superstaq.resource_estimate(circuit_random, "neutral_atom_qpu")
print(resource_estimate)

ResourceEstimate(num_single_qubit_gates=13, num_two_qubit_gates=4, depth=6)


# Example on Circuit Compiled on IBMQ Device

In [7]:
circuit2 = qiskit.QuantumCircuit(2)
circuit2.swap(0, 1)
circuit2.draw()

q_0: ─X─
      │ 
q_1: ─X─

In [8]:
out = superstaq.ibmq_compile(circuit2, target="ibmq_jakarta_qpu")
print(out.circuit)

     ┌─────────┐┌───────────┐┌─────────┐ ┌─────────┐┌───────┐┌───────────┐»
q_0: ┤ Rz(π/2) ├┤0          ├┤ Rz(π/2) ├─┤ Rx(π/2) ├┤ Rz(π) ├┤0          ├»
     └─────────┘│  Acecr_mp │├─────────┴┐├─────────┤└───────┘│  Acecr_pm │»
q_1: ───────────┤1          ├┤ Rz(3π/2) ├┤ Rx(π/2) ├─────────┤1          ├»
                └───────────┘└──────────┘└─────────┘         └───────────┘»
q_2: ─────────────────────────────────────────────────────────────────────»
                                                                          »
q_3: ─────────────────────────────────────────────────────────────────────»
                                                                          »
q_4: ─────────────────────────────────────────────────────────────────────»
                                                                          »
q_5: ─────────────────────────────────────────────────────────────────────»
                                                                          »
q_6: ───────

In [9]:
resource_estimate = superstaq.resource_estimate(circuit2, "ibmq_jakarta_qpu")
print(resource_estimate)

ResourceEstimate(num_single_qubit_gates=10, num_two_qubit_gates=3, depth=10)


# Multiple Circuit Example

In [10]:
# Create List of random circuits (3)
circuits = [
    random_circuit(2, 2, measure=True),
    random_circuit(3, 3, measure=True),
    random_circuit(4, 4, measure=True),
]
for c in circuits:
    print(c)

          ┌───┐┌─┐   
q_0: ──■──┤ Y ├┤M├───
     ┌─┴─┐├───┤└╥┘┌─┐
q_1: ┤ Y ├┤ H ├─╫─┤M├
     └───┘└───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 
                 ┌──────────┐┌───────────────────────────┐┌─┐      
q_0: ───■────────┤ Rz(3.59) ├┤ U3(0.70799,1.9078,5.5128) ├┤M├──────
      ┌─┴─┐ ┌───┐└────┬─────┘└───────────┬───┬───────────┘└╥┘┌─┐   
q_1: ─┤ H ├─┤ T ├─────┼──────────────────┤ H ├─────────────╫─┤M├───
     ┌┴───┴┐└───┘     │                 ┌┴───┴┐            ║ └╥┘┌─┐
q_2: ┤ Sdg ├──────────■─────────────────┤ Tdg ├────────────╫──╫─┤M├
     └─────┘                            └─────┘            ║  ║ └╥┘
c: 3/══════════════════════════════════════════════════════╩══╩══╩═
                                                           0  1  2 
     ┌───┐┌────────────┐┌────────────┐ ┌────────────┐              ┌─┐      
q_0: ┤ X ├┤ Rz(3.5348) ├┤ Rz(3.1407) ├─┤ Rx(2.9763) ├──────────────┤M├──────
     └─┬─┘└────────────┘└────────────┘┌┴────────────┤              └╥┘┌─┐   
q_1

In [11]:
resource_estimates = superstaq.resource_estimate(circuits, "neutral_atom_qpu")
print(resource_estimates)

[ResourceEstimate(num_single_qubit_gates=4, num_two_qubit_gates=1, depth=3), ResourceEstimate(num_single_qubit_gates=8, num_two_qubit_gates=2, depth=4), ResourceEstimate(num_single_qubit_gates=12, num_two_qubit_gates=1, depth=5)]
